In [ ]:
# C1 — Imports + helper (image preview)

import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from roboflow import Roboflow
import supervision as sv

def show_bgr(img_bgr, title=None, size=(6,6)):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=size)
    if title:
        plt.title(title)
    plt.axis("off")
    plt.imshow(img_rgb)
    plt.show()

In [ ]:
# C2 — Load API key and config from .env

load_dotenv()

API_KEY = os.getenv("ROBOFLOW_API_KEY")
MODEL_ID_RAW = os.getenv("ROBOFLOW_MODEL_ID")  # e.g. "applefinder-im9ep/2"
CONFIDENCE = int(os.getenv("CONFIDENCE", "50"))
OVERLAP = int(os.getenv("OVERLAP", "50"))
WORKSPACE = os.getenv("ROBOFLOW_WORKSPACE")  # optional

if not API_KEY:
    raise RuntimeError("ROBOFLOW_API_KEY not set")

if not MODEL_ID_RAW or "/" not in MODEL_ID_RAW:
    raise RuntimeError("ROBOFLOW_MODEL_ID must look like 'project-slug/version'")

PROJECT_SLUG, VERSION_STR = MODEL_ID_RAW.split("/", 1)
VERSION = int(VERSION_STR)

In [ ]:
# C3 — Connect to Roboflow

rf = Roboflow(api_key=API_KEY)
ws = rf.workspace(WORKSPACE) if WORKSPACE else rf.workspace()
project = ws.project(PROJECT_SLUG)
model = project.version(VERSION).model

In [ ]:
# C4 — Run inference on an image

image_path = "sample.jpg"
result = model.predict(image_path, confidence=CONFIDENCE, overlap=OVERLAP).json()

In [ ]:
# C5 — Predictions to DataFrame

preds = result["predictions"]
df = pd.DataFrame(preds)
df

In [ ]:
# C6 — Save results

out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

df.to_csv(out_dir / "results.csv", index=False)
print("Results saved in outputs/results.csv")

In [ ]:
# C7 — Visualize predictions

image = cv2.imread(image_path)

# Create detections (handle different supervision versions)
try:
    detections = sv.Detections.from_roboflow(result)
except AttributeError:
    detections = sv.Detections.from_inference(result)

box_annotator = sv.BoxAnnotator()
labels = [f"{pred['class']} {pred['confidence']:.2f}" for pred in preds]

annotated = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)
cv2.imwrite(str(out_dir / "annotated.jpg"), annotated)
print("Saved annotated image to outputs/annotated.jpg")
show_bgr(annotated, "Predictions")

In [ ]:
# C8 — Confidence histogram

plt.figure(figsize=(6,4))
df["confidence"].hist(bins=10)
plt.title("Confidence distribution")
plt.xlabel("Confidence")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(str(out_dir / "confidence_hist.png"), dpi=150)
plt.show()

In [ ]:
# C9 — Class counts

df["class"].value_counts().plot(kind="bar")
plt.title("Predicted objects count by class")
plt.xlabel("Class")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(str(out_dir / "class_counts.png"), dpi=150)
plt.show()

In [ ]:
# C10 — Export selected assets to docs/ for GitHub
from pathlib import Path
import shutil

docs_dir = Path("docs")
docs_dir.mkdir(exist_ok=True)

assets = [
    out_dir / "annotated.jpg",
    out_dir / "confidence_hist.png",
    out_dir / "class_counts.png",
]

for asset in assets:
    if asset.exists():
        shutil.copy(asset, docs_dir / asset.name)
        print(f"Copied {asset.name} -> docs/{asset.name}")
    else:
        print(f"Skipping {asset.name} (not found)")